In [1]:
NUM_STOCKS=2
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

import concurrent.futures

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# getting data

things we did to clean data:

1. shamelessly copy:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

2. we removed the tickers with $

3. we added exception handeling that tries similer strings to the input (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)
4.  we just drop exceptions that are found when getting the data from online  (these are diffrent every day)

# how are we keeping consistency? 
since the avilble stocks may change depending on the source of the data some extra steps need to be done in order to
alow saves to work seemlesly 


basic plan:

1. clean the data as much as possible 

2. save the embedings weights with their ticker string

3. backup everything on git to alow us to fix it



In [2]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

In [3]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
sav_set.discard("") 

ticker_list=list(sav_set)

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1582 unqualified stock symbols...
There are 10623 qualified stock symbols...


In [4]:
start='01/01/2020' 
end='09/14/2022'

In [5]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()

def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [6]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
error number 1: name=OAIM
100
200
error number 2: name=ATEST.A
300
400
error number 3: name=ZEXIT
500
error number 4: name=CTEST.E
600
error number 5: name=GGLS
700
800
error number 6: name=ZAZZT
900
1000
error number 7: name=AMPX
1100
error number 8: name=ZVZZC
1200
1300
1400
1500
error number 9: name=ZTEST
1600
error number 10: name=CTEST
error number 11: name=CTEST.V
1700
1800
1900
2000
2100
2200
2300
error number 12: name=XHLF
2400
2500
error number 13: name=CTEST.G
2600
2700
2800
error number 14: name=ZWZZT
2900
3000
error number 15: name=ZXZZT
3100
error number 16: name=IBO
3200
3300
error number 17: name=HOLO
3400
error number 18: name=ZIEXT
3500
3600
3700
3800
3900
4000
4100
4200
4300
error number 19: name=CTEST.O
4400
4500
error number 20: name=RCA
4600
error number 21: name=OAEM
error number 22: name=ZVZZT
4700
4800
error number 23: name=ZXIET
4900
5000
5100
5200
error number 24: name=ZVV
error number 25: name=XFIV
5300
error number 26: name=NXL
error 

In [7]:
data_df=pd.DataFrame(data).transpose()
data_df

,QQQJ,NZUS,GBNY,UCTT,AN,IBER.U,TGH,TMFX,DWMC,EFAD,...,QQQE,DAO,TENG,CTRN,TPGY.U,RWK,SMLF,GRZZ,OXM,AAPU
2020-01-02,NaN,NaN,NaN,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,...,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,NaN,1.000000,NaN
2020-01-03,NaN,NaN,NaN,1.005625,0.982012,NaN,0.992488,NaN,1.001264,0.987478,...,0.990847,0.956667,NaN,1.003090,NaN,0.991850,0.997341,NaN,0.995559,NaN
2020-01-06,NaN,NaN,NaN,0.944180,0.965249,NaN,0.987793,NaN,1.003793,0.991457,...,0.991565,0.994667,NaN,1.003532,NaN,0.990774,0.996968,NaN,1.003918,NaN
2020-01-07,NaN,NaN,NaN,0.972739,0.949918,NaN,1.009390,NaN,1.003203,0.988894,...,0.995872,1.062000,NaN,0.988079,NaN,0.991389,0.993469,NaN,1.000392,NaN
2020-01-08,NaN,NaN,NaN,0.965816,0.943581,NaN,1.008451,NaN,1.012432,0.992433,...,1.001795,1.048000,NaN,0.988521,NaN,0.991081,0.997551,NaN,0.987330,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,0.942622,0.902038,1.092170,1.211597,2.441742,1.024871,2.856008,0.750576,1.328417,0.862245,...,1.202725,0.354667,0.679256,0.956630,1.024590,1.311733,1.167402,0.919085,1.229748,0.914537
2022-09-08,0.955089,0.908192,1.100099,1.245781,2.403516,1.024871,2.846327,0.760150,1.318134,0.863289,...,1.215767,0.344667,0.687124,0.936682,1.024590,1.319384,1.173824,0.929194,1.235751,0.903084
2022-09-09,0.974735,0.923153,1.091179,1.287321,2.402085,1.024871,2.987675,0.776541,1.343864,0.883427,...,1.244386,0.362667,0.713443,0.953971,1.024590,1.349519,1.196658,0.948060,1.256624,0.927914
2022-09-12,0.980402,0.932042,1.096135,1.273042,2.384914,1.024871,3.013815,0.785213,1.351905,0.892491,...,1.255616,0.379333,0.725166,0.931806,1.025615,1.360917,1.211881,0.958169,1.259625,0.980777


In [8]:
train_df=data_df.iloc[:-68]
test_df=data_df.iloc[-68:]

test_df

,QQQJ,NZUS,GBNY,UCTT,AN,IBER.U,TGH,TMFX,DWMC,EFAD,...,QQQE,DAO,TENG,CTRN,TPGY.U,RWK,SMLF,GRZZ,OXM,AAPU
2022-06-07,0.982202,0.924610,1.150644,1.441800,2.502249,1.019689,3.222336,0.759499,1.522242,0.924454,...,1.258333,0.346667,0.849876,1.200443,1.014344,1.410257,1.282049,1.015801,1.257988,NaN
2022-06-08,0.975790,0.913471,1.153617,1.440069,2.480376,1.019689,3.066788,0.745865,1.503875,0.914241,...,1.250182,0.450667,0.822062,1.212412,1.015369,1.383401,1.260880,1.004935,1.253486,NaN
2022-06-09,0.947501,0.893115,1.154608,1.412808,2.509403,1.016580,3.019740,0.734637,1.461302,0.900531,...,1.215404,0.370000,0.791770,1.192907,1.014344,1.361386,1.233050,0.976995,1.315697,NaN
2022-06-10,0.915440,0.865988,1.154608,1.356556,2.455438,1.016580,2.963089,0.711779,1.448817,0.883199,...,1.178272,0.316667,0.746450,1.125526,1.015369,1.321414,1.200464,0.955502,1.281181,NaN
2022-06-13,0.864142,0.830206,1.154608,1.253137,2.277392,1.016580,2.752812,0.674236,1.361385,0.863075,...,1.122302,0.315333,0.672725,1.075877,1.015369,1.255835,1.147422,0.902129,1.211058,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,0.942622,0.902038,1.092170,1.211597,2.441742,1.024871,2.856008,0.750576,1.328417,0.862245,...,1.202725,0.354667,0.679256,0.956630,1.024590,1.311733,1.167402,0.919085,1.229748,0.914537
2022-09-08,0.955089,0.908192,1.100099,1.245781,2.403516,1.024871,2.846327,0.760150,1.318134,0.863289,...,1.215767,0.344667,0.687124,0.936682,1.024590,1.319384,1.173824,0.929194,1.235751,0.903084
2022-09-09,0.974735,0.923153,1.091179,1.287321,2.402085,1.024871,2.987675,0.776541,1.343864,0.883427,...,1.244386,0.362667,0.713443,0.953971,1.024590,1.349519,1.196658,0.948060,1.256624,0.927914
2022-09-12,0.980402,0.932042,1.096135,1.273042,2.384914,1.024871,3.013815,0.785213,1.351905,0.892491,...,1.255616,0.379333,0.725166,0.931806,1.025615,1.360917,1.211881,0.958169,1.259625,0.980777


In [9]:
ticker_list=data_df.columns
print(f"{len(symbols)-len(ticker_list)} stocks were droped")

1644 stocks were droped


# preprocessing 


1. ticker: this is also used to select which stocks come to the model

2. price: we take the ratio: (importantly we did drop some potentialy important indicators)

3. date: the day and month are floating point numbers. while year and day of the week are embedings.



In [70]:
look=layers.StringLookup(vocabulary=ticker_list,num_oov_indices=0)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True,num_oov_indices=0)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)

In [71]:
inputs=layers.Input([],dtype=tf.string)
x=look(inputs)
x=tick_embed(x)
ticker_model=tf.keras.Model(inputs,x)
ticker_model.compile()

In [72]:
dataset=tf.data.Dataset.from_tensor_slices(ticker_list)
exmple=dataset.batch(17).take(1).get_single_element()
exmple

<tf.Tensor: shape=(17,), dtype=string, numpy=
array([b'QQQJ', b'NZUS', b'GBNY', b'UCTT', b'AN', b'IBER.U', b'TGH',
       b'TMFX', b'DWMC', b'EFAD', b'PTC', b'CIX', b'AB', b'HYDB', b'TBB',
       b'SSUS', b'JHI'], dtype=object)>

In [73]:
#data_df[exmple.numpy().astype('U13')]

In [74]:
def thread_distribute(l,f,n=1):
    #print(type(l))
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [77]:
def choose_stocks(x,data_df=train_df):
    a=look(ticker_list)
    #print(a.shape)
    a=tf.one_hot(a,len(ticker_list)+1)
    #print(a.shape)
    a=a@ticker_model.weights[0]
    #b=tick_embed(look(x))
    #a=ticker_model(ticker_list)
    b=ticker_model(x)
    
    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,NUM_STOCKS)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    return [[data_df[x] for x in sub] for sub in tickers]
    #return thread_distribute(tickers,lambda x: data_df[x],2)

a=choose_stocks(exmple)

pd.DataFrame(a[0]).transpose().dropna()

,QQQJ,CID
2020-10-13,1.000000,0.795406
2020-10-14,0.995151,0.795646
2020-10-15,0.992167,0.785974
2020-10-16,0.991421,0.788978
2020-10-19,0.987318,0.785583
...,...,...
2022-05-31,0.963343,1.006195
2022-06-01,0.952027,1.003086
2022-06-02,0.982957,1.012412
2022-06-03,0.964097,1.017385


In [78]:
a[0][0].index

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26',
               '2022-05-27', '2022-05-31', '2022-06-01', '2022-06-02',
               '2022-06-03', '2022-06-06'],
              dtype='datetime64[ns]', length=612, freq=None)

In [79]:
def time_encode(a):
    #a=a.index
    #print(type(a))
    return (a.day_of_week,a.day/30,a.month/12,(a.year-2020))
    
time_encode(data_df.index[0])

(3, 0.06666666666666667, 0.08333333333333333, 0)

In [80]:
def choose_stocks_df(x,data_df=train_df):
    a=look(ticker_list)
    a=tf.one_hot(a,len(ticker_list)+1)
    a=a@ticker_model.weights[0]
    #a=ticker_model(ticker_list)
    b=look(x)
    b=tf.one_hot(b,len(ticker_list)+1)
    b=b@ticker_model.weights[0]

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,NUM_STOCKS)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    
    return [pd.DataFrame([data_df[x] for x in sub]).transpose().dropna() for sub in tickers]

v=choose_stocks_df(exmple)
v[0]

,QQQJ,CID
2020-10-13,1.000000,0.795406
2020-10-14,0.995151,0.795646
2020-10-15,0.992167,0.785974
2020-10-16,0.991421,0.788978
2020-10-19,0.987318,0.785583
...,...,...
2022-05-31,0.963343,1.006195
2022-06-01,0.952027,1.003086
2022-06-02,0.982957,1.012412
2022-06-03,0.964097,1.017385


# time handeling 

problems with the data in curent time format:

1. there are null entries especialy from before a company was made 
2. the time diffrence between entries isnt clean

In [81]:
mask=np.logical_not(data_df.isna().to_numpy())
mask

array([[False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False,  True, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [82]:
# need to add the tickers strings/ints
def get_xs(v):
    t=[x.index for x in v]
    #for x in t:
     #   print(type(x))
    t=thread_distribute(t,time_encode,2)
    #t=[[x[i] for x in t] for i in range(4)]
    print("time encoded")
    t=[thread_distribute(t,lambda x: x[i],2) for i in range(4)]
    #thread_distribute(t,len,2)
    t=[tf.ragged.constant(x) for x in t]
    
    x=thread_distribute(v,lambda x: x.to_numpy(),1)
    vals=tf.concat(x,axis=0)
    lengths=[len(s) for s in x]
    x=tf.RaggedTensor.from_row_lengths(vals,lengths)
    #[x.shape for x in x]
    
    "padding"
    x=(x+1).to_tensor()-1
    t=[(x+1).to_tensor()-1 for x in t] 
    
    x=tf.transpose(x,[2,0,1])
    t.extend(x) 
    
    tickers=tf.constant([x.transpose().index for x in v])
    t.append(tickers)
    return t

xs=get_xs(v)
[(x.shape,x.dtype) for x in xs]

time encoded


[(TensorShape([17, 612]), tf.int32),
 (TensorShape([17, 612]), tf.float32),
 (TensorShape([17, 612]), tf.float32),
 (TensorShape([17, 612]), tf.int32),
 (TensorShape([17, 612]), tf.float64),
 (TensorShape([17, 612]), tf.float64),
 (TensorShape([17, 2]), tf.string)]

In [83]:
#[len(x) for x in v]

In [84]:
def sample_seq(x,LEN=100):
    l=len(x)
    if(l<LEN):
        return x 
    
    start=np.random.randint(l-LEN)
    return x.iloc[start:start+LEN]
sample_seq(v[-1]) 
sample_seq(v[0])

,QQQJ,CID
2021-04-21,1.230839,0.998070
2021-04-22,1.232707,0.993356
2021-04-23,1.250643,0.998223
2021-04-26,1.266336,1.000764
2021-04-27,1.264842,0.997305
...,...,...
2021-09-03,1.321180,1.028296
2021-09-07,1.312579,1.028296
2021-09-08,1.299491,1.018162
2021-09-09,1.301360,1.013344


In [85]:
base=[sample_seq(x) for x in v]
y=[s.iloc[1:,0] for s in base]
x=[s[:-1] for s in base]
x=get_xs(x)
x

time encoded


[<tf.Tensor: shape=(17, 99), dtype=int32, numpy=
 array([[ 2,  3,  4, ...,  3,  4,  0],
        [ 0,  1,  2, ..., -1, -1, -1],
        [ 0,  1,  2, ...,  4,  1,  2],
        ...,
        [ 3,  4,  0, ...,  2,  3,  4],
        [ 0,  1,  2, ...,  4,  0,  1],
        [ 2,  3,  4, ...,  0,  1,  2]])>,
 <tf.Tensor: shape=(17, 99), dtype=float32, numpy=
 array([[ 0.13333333,  0.16666663,  0.20000005, ...,  0.83333325,
          0.8666667 ,  0.9666667 ],
        [ 0.6       ,  0.6333333 ,  0.66666675, ..., -1.        ,
         -1.        , -1.        ],
        [ 1.        ,  1.0333333 ,  0.0333333 , ...,  0.4666667 ,
          0.6       ,  0.6333333 ],
        ...,
        [ 1.        ,  0.0333333 ,  0.13333333, ...,  0.5333333 ,
          0.5666666 ,  0.6       ],
        [ 0.83333325,  0.8666667 ,  0.9       , ...,  0.36666667,
          0.4666667 ,  0.5       ],
        [ 0.29999995,  0.33333337,  0.36666667, ...,  0.83333325,
          0.8666667 ,  0.9       ]], dtype=float32)>,
 <tf.Te

In [86]:
def get_numbers(tickers):
    v=choose_stocks_df(tickers)
    base=[sample_seq(x) for x in v]
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_numbers(exmple)
print(time.time()-t)

time encoded
1.0308666229248047


In [50]:
specs=[tf.TensorSpec.from_tensor(x) for x in n]
s=[list(x.shape) for x in specs]
for x in s:
    x[0]=None
specs=[tf.TensorSpec(s[i],specs[i].dtype) for i in range(len(specs))]
specs

[TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.string, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None)]

In [51]:
def gather_x(*args):
    return args[:-1]

def gather_y(*args):
    return args[-1]

gather_x(1,2,3,4),gather_y(1,2,3,4)

((1, 2, 3), 4)

# train dataset 
because of the need to use the dataset as a refrence type in the pandas df we are forcing tensorflow to run the whole thing eagerly. 

it shouldnt be a major issue since most the heavy lifting is done in pandas which is optimized c code 

In [52]:
#need to do training and test set (test set should look diffrent)
train_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_numbers,inp=[y],Tout=specs))

dataset_x=train_dataset.map(gather_x)
dataset_y=train_dataset.map(gather_y)
train_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=train_dataset.take(1).get_single_element()
x

time encoded
time encoded


((<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[4, 0, 1, ..., 3, 4, 1],
         [2, 3, 4, ..., 1, 2, 3],
         [0, 1, 2, ..., 0, 1, 2],
         ...,
         [1, 2, 3, ..., 0, 1, 2],
         [1, 2, 3, ..., 4, 0, 1],
         [1, 2, 3, ..., 1, 2, 3]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[0.26666665, 0.36666667, 0.39999998, ..., 0.9       , 0.9333334 ,
          0.0333333 ],
         [0.16666663, 0.20000005, 0.23333335, ..., 0.70000005, 0.73333335,
          0.76666665],
         [0.66666675, 0.70000005, 0.73333335, ..., 0.29999995, 0.33333337,
          0.36666667],
         ...,
         [0.5333333 , 0.5666666 , 0.6       , ..., 0.06666672, 0.10000002,
          0.13333333],
         [0.20000005, 0.23333335, 0.26666665, ..., 0.66666675, 0.76666665,
          0.79999995],
         [0.39999998, 0.4333334 , 0.4666667 , ..., 0.0333333 , 0.06666672,
          0.10000002]], dtype=float32)>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  arr

In [53]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 2), dtype=tf.string, name=None)]

In [54]:
def cut23(x):
    return {"time":x[:4],"prices":x[4:NUM_STOCKS],"tickers":x[NUM_STOCKS]}

cut23(x[0])

{'time': (<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[4, 0, 1, ..., 3, 4, 1],
         [2, 3, 4, ..., 1, 2, 3],
         [0, 1, 2, ..., 0, 1, 2],
         ...,
         [1, 2, 3, ..., 0, 1, 2],
         [1, 2, 3, ..., 4, 0, 1],
         [1, 2, 3, ..., 1, 2, 3]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[0.26666665, 0.36666667, 0.39999998, ..., 0.9       , 0.9333334 ,
          0.0333333 ],
         [0.16666663, 0.20000005, 0.23333335, ..., 0.70000005, 0.73333335,
          0.76666665],
         [0.66666675, 0.70000005, 0.73333335, ..., 0.29999995, 0.33333337,
          0.36666667],
         ...,
         [0.5333333 , 0.5666666 , 0.6       , ..., 0.06666672, 0.10000002,
          0.13333333],
         [0.20000005, 0.23333335, 0.26666665, ..., 0.66666675, 0.76666665,
          0.79999995],
         [0.39999998, 0.4333334 , 0.4666667 , ..., 0.0333333 , 0.06666672,
          0.10000002]], dtype=float32)>,
  <tf.Tensor: shape=(32, 99), dtype=float32, nump

# validation dataset 
on infrance the best results would probably be achived by runing on every time entry with at least some memory of the past

that means there would need to be some precomputation which we can make diffrent for timesetep. 

this is rediclously slow and combersom to write so our validation is just gona look mostly like the training. 
after choosing the stocks we would want to work with we are going to think of test behivior 

we may want to just train a new model from scratch only on that subset.

In [55]:
def get_val_numbers(tickers):
    base=choose_stocks_df(tickers,test_df)
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_val_numbers(exmple)
print(time.time()-t)

time encoded
0.9255223274230957


In [56]:
#need to do training and test set (test set should look diffrent)
val_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_val_numbers,inp=[y],Tout=specs))

dataset_x=val_dataset.map(gather_x)
dataset_y=val_dataset.map(gather_y)
val_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=val_dataset.take(1).get_single_element()

time encoded
time encoded


In [57]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(32, 67), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 67), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 67), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 67), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 67), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 67), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 2), dtype=tf.string, name=None)]

# the model

**the dataset is dependent on the model!!!** this means we have some considerations on how it should look like:

* there has to be an attention mechanisem on embeding space. (using only the sampling embedings dot product) 
* embedings convergnce and stabilety greatly effects the models training. (we may want to consider reverse regularization)
*  at least some part of the model should be seprable by stock, in order to reduce noise from data sampeling
* it would defintly help if we add residual conections to the embedings to alow gradients to flow to the embedings

*side note: the padding used is -1, seems like it should propgate to the loss*


In [89]:
#setting up the inputs
inputs=[layers.Input([None,],dtype=x.dtype) for x in specs[:-2]]
inputs.append(layers.Input(specs[-2].shape[1:],dtype=specs[-2].dtype,name="tickers"))

#time encoding
time_depndent=[tf.expand_dims(x,-1) for x in inputs[:-1]] 
time_depndent=[layers.Masking(-1)(x) for x in time_depndent] 


day_embeding=layers.Embedding(7,7)(time_depndent[0])
year_embeding=layers.Embedding(3,5)(time_depndent[3]) 

day_embeding=layers.TimeDistributed(layers.Reshape([7]))(day_embeding)
year_embeding=layers.TimeDistributed(layers.Reshape([5]))(year_embeding)

t=tf.concat(time_depndent[1:3],-1)
t=layers.Dense(20)(t) 

time_encode=tf.concat([day_embeding,t,year_embeding],-1)
time_encode

<KerasTensor: shape=(None, None, 32) dtype=float32 (created by layer 'tf.concat_3')>

In [96]:
#tickers=look(inputs[-1])
#tickers=tick_embed(tickers)
tickers=layers.Reshape([NUM_STOCKS,1])(inputs[-1])
tickers=layers.TimeDistributed(ticker_model)(tickers)
tickers=layers.Reshape([NUM_STOCKS,512])(tickers)
origin=tf.expand_dims(tickers[:,0],-2)
origin=layers.Permute([2,1])(origin)
attention=tickers@origin
attention=layers.Permute([2,1])(attention)
attention=tf.nn.softmax(attention)
attention

<KerasTensor: shape=(None, 1, 2) dtype=float32 (created by layer 'tf.nn.softmax_2')>

In [97]:
comp=layers.Dense(33)(tickers)
comp

<KerasTensor: shape=(None, 2, 33) dtype=float32 (created by layer 'dense_4')>

In [98]:
"need to add stock conditioning"
prices=time_depndent[4:4+NUM_STOCKS]
s=[layers.Concatenate(-1)([time_encode,x]) for x in prices]

#adding ticker information (while not alowing it to effect the mask)
masks=[x._keras_mask for x in s] 
s=[layers.Add()([s[i],comp[:,i]]) for i in range(len(s))]

stock_layer=layers.LSTM(16,return_sequences=True)
s=[stock_layer(s[i],mask=masks[i]) for i in range(len(s))]
stock_layer=layers.GRU(16,return_sequences=True)
s=[stock_layer(x) for x in s]

s=[tf.expand_dims(x,-2) for x in s]
s=layers.Concatenate(-2)(s)
s

<KerasTensor: shape=(None, None, 2, 16) dtype=float32 (created by layer 'concatenate_8')>

In [99]:
sumorized=attention@tf.transpose(s,[1,0,2,3])
sumorized=tf.transpose(sumorized,[1,0,2,3])
sumorized=layers.TimeDistributed(layers.Reshape([16]))(sumorized)
sumorized

<KerasTensor: shape=(None, None, 16) dtype=float32 (created by layer 'time_distributed_8')>

In [100]:
x=layers.LSTM(8,return_sequences=True,)(sumorized)
x=layers.LayerNormalization()(x)
x=layers.Permute([2,1])(x)
x=layers.GlobalMaxPool1D()(x)
x

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'global_max_pooling1d_1')>

In [101]:
model=tf.keras.Model(inputs,x)
model.compile("adam","mse")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                            

                                                                                                  
 add_4 (Add)                    (None, None, 33)     0           ['concatenate_6[0][0]',          
                                                                  'tf.__operators__.getitem_8[0][0
                                                                 ]']                              
                                                                                                  
 add_5 (Add)                    (None, None, 33)     0           ['concatenate_7[0][0]',          
                                                                  'tf.__operators__.getitem_9[0][0
                                                                 ]']                              
                                                                                                  
 tf.expand_dims_19 (TFOpLambda)  (None, 1, 512)      0           ['tf.__operators__.getitem_7[0][0
          

In [103]:
numbers=get_numbers(exmple.numpy())
x=numbers[:-1]
y=numbers[-1]

TypeError: 'KerasTensor' object is not callable

In [102]:
model.train_on_batch(x,y)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
a=tf.ones([17,1,2])
b=tf.ones([99,17,2,16])

a@b

In [191]:
ticker_model.weights[0].numpy()

array([[-0.00819845,  0.03138752, -0.04917015, ..., -0.00844888,
        -0.02786379, -0.03285414],
       [ 0.01930871,  0.00275303,  0.02794688, ...,  0.02111563,
         0.04424438,  0.01380203],
       [ 0.02344929, -0.04172752, -0.02175329, ...,  0.04957949,
        -0.03892108,  0.03528101],
       ...,
       [-0.02316197, -0.02107832,  0.00546788, ...,  0.03915573,
        -0.02534916,  0.03187651],
       [-0.00625204,  0.0424701 ,  0.03526393, ...,  0.01707247,
         0.04708024,  0.04816936],
       [-0.01116301,  0.02012744, -0.01742548, ..., -0.00975705,
        -0.03553439, -0.04758307]], dtype=float32)

In [202]:
ticker_model(exmple)[0][0]

<tf.Tensor: shape=(), dtype=float32, numpy=-0.008198451>

In [201]:
look(exmple)[0]

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [203]:
ticker_model.weights[0].numpy()[0][0]

-0.008198451

In [205]:
ticker_model.weights[0]

<tf.Variable 'embedding_14/embeddings:0' shape=(10430, 512) dtype=float32, numpy=
array([[-0.00819845,  0.03138752, -0.04917015, ..., -0.00844888,
        -0.02786379, -0.03285414],
       [ 0.01930871,  0.00275303,  0.02794688, ...,  0.02111563,
         0.04424438,  0.01380203],
       [ 0.02344929, -0.04172752, -0.02175329, ...,  0.04957949,
        -0.03892108,  0.03528101],
       ...,
       [-0.02316197, -0.02107832,  0.00546788, ...,  0.03915573,
        -0.02534916,  0.03187651],
       [-0.00625204,  0.0424701 ,  0.03526393, ...,  0.01707247,
         0.04708024,  0.04816936],
       [-0.01116301,  0.02012744, -0.01742548, ..., -0.00975705,
        -0.03553439, -0.04758307]], dtype=float32)>